In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
import pyspark.sql.functions as f

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Spark_movies_task") \
    .config("spark.master", "spark://ZHUYANZHAO-HWPC.localdomain:7077") \
    .config("spark.jars.packages", "mysql:mysql-connector-java:8.0.22") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.num", "1") \
    .getOrCreate()

In [ ]:
df_movies = spark.read.option("header","true").csv("./movies_metadata.csv")
df_movies = df_movies.withColumnRenamed("id", "tmdb_id") \
.withColumn('imdb_id', f.col('imdb_id').substr(f.lit(4), f.length('imdb_id'))) \
.drop('belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages') \
.na.drop(subset=['tmdb_id'])
df_movies.printSchema()
df_movies.show(5)

In [ ]:
df_movies.select(*df_movies.columns).write.format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/spark_movies") \
    .option("driver", "com.mysql.cj.jdbc.Driver").option("dbtable", "movies") \
    .option("user", "root").option("password", "zhuyz928").save()